# Kotlin programming

## Find sublist

This function should return a list containing only the numbers that equal or exceed the threshold.

In [1]:
fun pruneList(list: List<Int>, threshold: Int): List<Int> = list.filter { it >= threshold }

require(pruneList(listOf(1, 2, 3, 4, 5, 6), 5) == listOf(5, 6))

## Find smallest element

This function should find the smallest element in the list:

In [2]:
fun smallest(list: List<Int>): Int = list.min()

require(smallest(listOf(5, 3, 2, 4)) == 2)

## Find stats

The `stats` function should return more general stats about the list:

* Min value
* Max value
* Sum of all values
* Mean of all values

We have a `Stats` interface which captures all of these:

In [3]:
import java.math.BigInteger
import kotlin.collections.fold

interface Stats {
    val min: Int
    val max: Int
    val sum: Long
    val mean: Double
}

fun stats(list: List<Int>): Stats = object : Stats {
    override val min = list.min()
    override val max = list.max()
    override val sum = list.sumOf { it.toLong() }
    override val mean = sum.toDouble() / list.size;
}

val stats = stats(listOf(1, 2, 3, 4, 5))

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15L)

## Find stats with provider

Did the previous solution make multiple passes through the list?  Or did it rely on sorting the input? If so, we might need a re-think.

It turns out the function now needs to handle a case where the list of numbers may be too big to hold in memory. Instead
of passing the list, we instead pass a function that returns the next element in a (possibly long) series of numbers.

When the series is complete, the function returns `null`, therefore the return type of this function is `() -> Int?`.

### Subtask: Test support

For testing purposes, we need to write small utility function that converts a list into this kind of function. Just for show, we make it an extension function:

In [4]:
fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val provider = listOf(1, 2, 3).provider()

require(provider() == 1)
require(provider() == 2)
require(provider() == 3)
require(provider() == null)

### Main task

This revised edition should handle abitrarily long "lists", and it relies on our `provider` function for testing purposes:Here is a revised edition which can handle abitrarily long "lists", and which relies on our `provider` function for testing purposes:

In [5]:
`import java.math.BigInteger

interface Stats {
    val min: Int
    val max: Int
    val sum: BigInteger
    val mean: Double
}

data class StatsImpl(
    override val min: Int,
    override val max: Int,
    override val sum: BigInteger,
    val count: Int
) : Stats {

    constructor() : this(Int.MAX_VALUE, Int.MIN_VALUE, BigInteger.ZERO, 0)

    override val mean: Double
        get() = count.takeIf { it > 0 }
            ?.let { (sum.div(it.toBigInteger())).toDouble() }
            ?: 0.toDouble()

    operator fun plus(i: Int) = StatsImpl(
        min(this.min, i),
        max(this.max, i),
        this.sum + i.toBigInteger(),
        this.count + 1
    )
}

fun stats(list: () -> Int?): Stats =
    generateSequence(list).fold(StatsImpl()) { stats, integer ->
        stats + integer
    }

fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val provider = listOf(1, 2, 3, 4, 5).provider()

val stats = stats(provider)

require(stats.min == 1)
require(stats.max == 5)
require(stats.mean == 3.0)
require(stats.sum == 15.toBigInteger())

## Provide a hook

In another bizarre twist of fate, it now turns out that our stream processing engine is noticed by our colleagues.  They see it as a stable and mature infrastructure, and they want to "hook into" our code with completely arbitary processing logic.  Make it generic!

The below example is a simple "sum" function which hooks into our wonderful machine, generically named `process`:

In [6]:
fun <S> process(list: () -> Int?, initial: () -> S, processor: (S, Int) -> S): S =
    generateSequence(list).fold(initial(), processor)

fun List<Int>.provider(): () -> Int? = iterator().run {
    { takeIf { hasNext() }?.let { next() } }
}

val initial = { 0 }
val processor: (Int, Int) -> Int = { i1, i2 -> i1 + i2 }
val provider = listOf(1, 2, 3, 4, 5).provider()

require(process(provider, initial, processor) == 15)
